<a href="https://colab.research.google.com/github/Box-01/Coursera_Capstone/blob/master/Capstone_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h1 align="center"><font size="5">Capstone Project</font></h1>


This notebook will be used for the Coursera Capstone Project. This project is part of the **Applied Data Science Capstone Course** of IBM Data Science Certificate.

# Set-UP

In [0]:
import pandas as pd
import numpy as np

In [2]:
print("Hello Capstone Project Course!")

Hello Capstone Project Course!
